In [0]:
!wget https://fmi-unibuc-ia.github.io/ia/Data/data_lab5.zip

--2020-04-30 07:45:18--  https://fmi-unibuc-ia.github.io/ia/Data/data_lab5.zip
Resolving fmi-unibuc-ia.github.io (fmi-unibuc-ia.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fmi-unibuc-ia.github.io (fmi-unibuc-ia.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503740 (492K) [application/zip]
Saving to: ‘data_lab5.zip.1’

data_lab5.zip.1     100%[===================>] 491.93K  --.-KB/s    in 0.06s   

2020-04-30 07:45:18 (7.91 MB/s) - ‘data_lab5.zip.1’ saved [503740/503740]



In [0]:
!unzip data_lab5.zip

Archive:  data_lab5.zip
replace data/test_labels.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/test_labels.npy    
replace data/test_sentences.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace data/test_sentences.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/test_sentences.npy  
  inflating: data/training_labels.npy  
  inflating: data/training_sentences.npy  


In [0]:
import numpy as np

train_data = np.load('data/training_sentences.npy', allow_pickle=True)
train_labels = np.load('data/training_labels.npy')

test_data = np.load('data/test_sentences.npy', allow_pickle=True)
test_labels = np.load('data/test_labels.npy')

In [0]:
from sklearn import preprocessing

def normalize_data(train_data, test_data, norm_type=None):
    if norm_type is None:
        return train_data, test_data
    if norm_type == 'standard':
        scaler = preprocessing.StandardScaler()
        scaler.fit(train_data)
        return scaler.transform(train_data), scaler.transform(test_data)
    if norm_type == 'min_max':
        return (preprocessing.minmax_scale(train_data, axis=-1),
                preprocessing.minmax_scale(test_data, axis=-1))
    if norm_type == 'l1':
        return (preprocessing.normalize(train_data, norm='l1'),
                preprocessing.normalize(test_data, norm='l1'))
    if norm_type == 'l2':
        return (preprocessing.normalize(train_data, norm='l2'),
                preprocessing.normalize(test_data, norm='l2'))

np.random.seed(7)
normalize_data(np.random.randn(1, 7), np.random.randn(1, 5), norm_type='l2')

(array([[ 8.59950904e-01, -2.37016960e-01,  1.66952383e-02,
          2.07298827e-01, -4.01316034e-01,  1.05073308e-03,
         -4.52929004e-04]]),
 array([[-0.7930376 ,  0.45992471,  0.27139187, -0.28265904, -0.07753025]]))

In [0]:
class BagOfWords:
    def __init__(self):
        self.vocabulary = dict()
        self.known_words = ['UNK']

    @property
    def vocab_size(self):
        return len(self.known_words)

    def add_to_vocabulary(self, word):
        if word in self.vocabulary.keys():
            return

        self.vocabulary[word] = len(self.known_words)
        self.known_words.append(word)

    def build_vocabulary(self, data):
        for message in data:
            for word in message:
                self.add_to_vocabulary(word)

    def get_features(self, data):
        features = []
        for message in data:
            words = np.zeros(self.vocab_size)
            for word in message:
                index = self.vocabulary.get(word, 0)
                words[index] += 1
            features.append(words)

        return np.array(features)

bow = BagOfWords()
bow.build_vocabulary(train_data)

In [0]:
train_features = bow.get_features(train_data)
test_features = bow.get_features(test_data)

In [0]:
X_train, X_test = normalize_data(train_features, test_features)

In [0]:
from sklearn.svm import SVC

clf = SVC(C=1.0, kernel='linear')
clf.fit(X_train, train_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_preds = clf.predict(X_test)

In [0]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(test_labels, y_preds)
f1 = f1_score(test_labels, y_preds)

print("Accuracy:", accuracy)
print("F1 score:", f1)

Accuracy: 0.9847826086956522
F1 score: 0.9423868312757202


In [0]:
word_vectors = []

for word in bow.known_words[1:]:
    word_vector = np.zeros(bow.vocab_size)
    word_vector[bow.vocabulary[word]] = 1
    word_vectors.append(word_vector)

word_vectors = np.array(word_vectors)

values = clf.decision_function(word_vectors)

In [0]:
sorted_indexes = np.argsort(values)

top_positive = sorted_indexes[:10]
top_negative = sorted_indexes[-10:]

In [0]:
print("Top 10 spam:")
for word_index in top_positive:
    print(bow.known_words[word_index], end=' ')

print()

print("Top 10 non-spam:")
for word_index in top_negative:
    print(bow.known_words[word_index], end=' ')

print()

Top 10 spam:
wish contact httptms RingtoneFrom 85233 dado weekends TONESReply Ref9280114 ringtoneking 
Top 10 non-spam:
usc call message ryans thing one bread fringe legs Bergkamp 
